In [4]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4    import BeautifulSoup
from IPython.display import Markdown, display, update_display

from openai import OpenAI

In [3]:
# initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print('API key looks good so far')
else:
    print("There might be a problem with your API key.")

MODEL = 'gpt-4o-mini'
openai = OpenAI()
print(api_key)

API key looks good so far


In [5]:
# A class to present a page
class Website:
    """
    A utility class to represent a website that we have scrapped, now with links
    """
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else 'No title found'
        if soup.body:
            for irrelevant in soup.body(["script","style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n",strip=None)
        else:
            self.text =""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage title:\n{self.title}\nWebpage contents:\n{self.text}\n\n"
    

In [6]:
ed = Website('https://www.anthropic.com/')
print(ed.get_contents())

Webpage title:
Home \ Anthropic
Webpage contents:
Skip to main content
Skip to footer
Claude
API
Research
Commitments
Learn
News
Try Claude
AI 
research
 and 
products
 that put safety at the frontier
Claude.ai
Meet Claude 3.7 Sonnet
Claude 3.7 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
Claude 3.7 Sonnet and Claude Code
Introducing Claude 3.7 Sonnet, our most intelligent model yet and the first hybrid reasoning model. We’re also launching Claude Code, an agentic tool for coding. 
Read the announcement
Model details
Claude 3.7 Sonnet 
Research insights
Claude's extended thinking
Our Work
Product
Claude for Enterprise
Sep 4, 2024
Alignment
  ·  
Research
Constitutional AI: Harmlessness from AI Feedback
Dec 15, 2022
Announcements
Core Views on AI Safety: When, Why, What, and How
Mar 8, 2023
Work with Anthropic
Anthropic is an AI safety and research company base

## Step 1: Figure out relavant links with GPT-4o mini
Call to GPT-4o mini to read links on a webpage and respond in structured JSON

In [8]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a broucheer about the company. \
such as links to an About page or a Company page or Careers/Jobs page.\n"

link_system_prompt += "You should be respond in JSON as in this example:"
link_system_prompt += """
{
   "links":[
       {"type":"about page","url":"https://ful.url/goes/here/about"},
       {"type":"careers page","url":"https://another.ful.url/careers"},
   ]
}
"""

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a broucher about the company, repsond with the full https URL. Do not include Terms of Services, privacy, email links.\n"
    user_prompt += "Links (some might be relevant links):\n"
    user_prompt += "\n".join(website.links)

    return user_prompt


In [9]:
def get_relevant_links(url):    
    website = Website(url)    
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system", "content": link_system_prompt},
            {"role":"user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type":"json_object"}
    )
    
    result = completion.choices[0].message.content
    return json.loads(result)

In [10]:
get_relevant_links("https://www.anthropic.com/")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'},
  {'type': 'enterprise page', 'url': 'https://www.anthropic.com/enterprise'},
  {'type': 'transparency page',
   'url': 'https://www.anthropic.com/transparency'},
  {'type': 'responsible scaling policy page',
   'url': 'https://www.anthropic.com/responsible-scaling-policy'},
  {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'},
  {'type': 'supported countries page',
   'url': 'https://www.anthropic.com/supported-countries'},
  {'type': 'solutions page',
   'url': 'https://www.anthropic.com/solutions/coding'}]}

## Second step: Make the broucher
Assemble all the details into another prompt to GPT-4o

In [11]:
def get_all_details(url):
    result = "Landing page: \n"
    result += Website(url).get_contents()
    links = get_relevant_links(url)
    # print("Found links: ", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()
    return result        

In [12]:
system_prompt ="You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short broucher about the company for prospective customers, investors and recuits. Respond in markdown.\
Include details of company culture, customers and careers/jobs ifyou have the information."

In [13]:
def get_broucher_user_prompt(company_name, url):
    user_prompt = f"Your are looking at a company called: {company_name}. \n"
    user_prompt += f"Here are the content of its landing page and other relevant pages, use this information to build a short broucher in markdown. \n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000]
    return user_prompt    

In [14]:
def stream_brouher(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system", "content": system_prompt},
            {"role":"user",   "content": get_broucher_user_prompt(company_name, url)}
        ],
        stream=True
    )
    response=""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        resppnse = response.replace("```","").replace("markdown","")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [15]:
stream_brouher("Anthropic","https://www.anthropic.com/")

# Anthropic: Building a Safer AI Future

## About Us

Anthropic is an AI safety and research company based in San Francisco, dedicated to creating reliable, interpretable, and steerable AI systems. We believe that AI has the potential to revolutionize our world, and we commit ourselves to building systems that can be relied upon and conducting research that addresses both the opportunities and risks associated with AI.

### Our Mission
Our goal is to develop frontier AI systems that are not only effective but also safe for humanity. By treating AI safety as a systematic science, we strive for a **"race to the top"** dynamic in AI development that prioritizes safety and benefit for all.

## Our Products

### Claude
Meet **Claude 3.7 Sonnet**—our most intelligent AI model to date. Claude not only excels at natural language processing but is also the first hybrid reasoning model we've developed. It is designed to understand complex tasks and interact in a conversational manner, making it a robust tool for businesses, nonprofits, and civil society groups.

#### APIs and Integrations
With **Claude API**, developers can create AI-powered applications, integrating our technology to transform user experiences.

## Company Culture

At Anthropic, we value collaboration across disciplines, incorporating experts from machine learning, physics, policy, and engineering. Our company culture is guided by principles that prioritize global good, transparency, user kindness, and the mission of safe AI development.

### Core Values
- **Act for the Global Good**: We are committed to decisions that maximize positive long-term outcomes for humanity.
- **Hold Light and Shade**: We recognize the dual nature of AI—its risks and benefits—and navigate these complexities with care.
- **Be Good to Our Users**: Kindness and generosity characterize our interactions with all stakeholders.
- **Ignite a Race to the Top on Safety**: We aim to set the industry standard for safety in AI development.
- **Do the Simple Thing that Works**: We pursue empirical approaches that balance simplicity with effectiveness.
- **Be Helpful, Honest, and Harmless**: We foster a high-trust atmosphere where open communication prevails.
- **Put the Mission First**: Our mission shapes our collective purpose and guides our work.

## Join Our Team

### Career Opportunities
We are always looking for talented individuals who share our vision of making AI safe. Our team is comprised of interdisciplinary experts who collaborate closely to drive forward our mission.

#### Benefits
We provide a comprehensive range of benefits that emphasize health, work-life balance, and career development, which includes:
- Comprehensive health, dental, and vision insurance.
- Generous parental leave policies and flexible paid time off.
- Competitive salary and equity packages, including optional equity donation matching.
- Robust retirement plans and wellness stipends.

### Our Hiring Philosophy
We encourage candidates with diverse backgrounds to apply, whether or not they have experience in machine learning. Our hiring process emphasizes analytical thinking and problem-solving skills over specific academic qualifications.

## Customers We Serve
Our tools and AI models serve various sectors, including businesses looking to innovate, nonprofits seeking efficient solutions, and governmental agencies aiming to enhance policy making with data-driven insights.

---

For more information or to explore career opportunities, please visit our website or reach out to us directly. Together, let’s shape the future of safe AI.